## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-07 19:43:37 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657982,35.2,1454648,77.7,1355009,72.4
Vcells,1221413,9.4,8388608,64.0,1975153,15.1


In [2]:
PARAM <- list()
PARAM$experimento <- "e007_01"
PARAM$semilla_primigenia <- 100343

In [3]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Traigo df ya modificado en workflow A

In [18]:
library(duckdb)
setwd("~/Project_Wednesday")

con <- dbConnect(duckdb::duckdb(), dbdir=":memory:")
dataset <- dbGetQuery(con, "SELECT * FROM read_parquet('df.parquet')")
dbDisconnect(con)

head(dataset)

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,⋯,tc_fechalta_menor_delta_1,tc_fechalta_menor_delta_2,tc_fechamora_mayor_delta_1,tc_fechamora_mayor_delta_2,tc_fechamora_menor_delta_1,tc_fechamora_menor_delta_2,tc_fechacierre_mayor_delta_1,tc_fechacierre_mayor_delta_2,tc_fechacierre_menor_delta_1,tc_fechacierre_menor_delta_2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,249350768,202101,1,0,0,55,320,2386.7957,32045.10,408.6254,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,249350768,202102,1,0,0,55,321,1936.1925,32343.07,366.5057,⋯,28,NA,NA,NA,NA,NA,-7,NA,-7,NA
3,249350768,202103,1,0,0,55,322,4333.1642,34091.88,2540.1972,⋯,31,59,NA,NA,NA,NA,4,-3,4,-3
4,249350768,202104,1,0,0,55,323,2394.1612,32654.36,202.5176,⋯,30,61,NA,NA,NA,NA,1,5,1,5
5,249350768,202105,1,0,0,55,324,693.1992,29791.08,-1311.8592,⋯,31,61,NA,NA,NA,NA,3,4,3,4
6,249350768,202106,1,0,0,55,325,2268.0905,28536.63,425.8723,⋯,30,61,NA,NA,NA,NA,-5,-2,-5,-2


## Modelado

### Optimizacion de Hipeparámetros

Este script no hace optimización de hiperparámetros
<br> **No** se llama a una Bayesian Optimization, ese paso se saltea.
<br> No hace falta hacer particiones <train, validate, test>,  tampoco se hace un k-fold cross validation

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los cuatro meses  {202101, 202102, 202103, 202104}, esta es una meticulosa decisión.
* Se hace un muy agresivo undersampling del **0.10** = 10% de la clase mayoritaria (los "CONTINUA" )
* Obviamente los datos donde se aplica el modelo es el mes  {202106}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.
* No se optmizan los hiperparámetros de LightGBM, sino que se llama a la libreria *zLightGBM*
* Para *zLightGBM*  se crean **50** canaritos, esta es una decisión enorme !

In [22]:
# training y future
Sys.time()

PARAM$train_final$meses <- c(202101, 202102, 202103, 202104)
PARAM$train_final$undersampling <- 0.10

PARAM$future <- c(202106)

[1] "2025-11-07 19:58:12 UTC"

### Final Training Strategy

Se entrena en los cuatro meses {202101, 202102, 202103, 202104}
<br>Se hace un muy agresivo undersampling del 10% de la clase mayoritaria (los "CONTINUA" )

In [25]:
# se filtran los meses donde se entrena el modelo final

library(data.table)
setDT(dataset)

dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

In [26]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [27]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model

In [28]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

Loading required package: zlightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘zlightgbm’”
Installing package into ‘/home/rivas_ae97/.local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: zlightgbm



[1] "2025-11-07 20:02:20 UTC"

Un hiperparámetro de  *zLightGBM* es la cantidad de canaritos, en este caso se decidió (con tests realizados en {202101, 202102} fijarlo en 50
<br> esta es una importante decisión que debe tomarse
<br> en la version actual de *zLightGBM* los canaritos deben agregarse por fuera al dataset, y mandatoriamente deben ser las primeras columnas del mismo
<br> durante noviembre, si hay suerte, se liberará una versiónn que lo haga automáticamente internamente en el código C++ de la librería

In [29]:
# canaritos
PARAM$qcanaritos <- 50

cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$qcanaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

[1] "2025-11-07 20:02:34 UTC"

In [30]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [31]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 71550 columnas 1157 


[1] "2025-11-07 20:02:42 UTC"

Nuevos hiperparámetros en  *zLightGBM*
* canaritos , entero >=0,  cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
* gradient_bound , numero real, >=0  cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo

En el caso que  canaritos==0  y  gradient_bound==0  entonces  xLightGBM se comporta exactamente igual a  LightGBM

Tener en cuenta que el hiperparámetro  min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
<br> realmente valdria la pena experimentar con distintos valores de  min_data_in_leaf, hay potencial de mejora !

In [32]:
# definicion de parametros, los viejos y los nuevos

PARAM$lgbm <-  list(
  boosting= "gbdt",
  objective= "binary",
  metric= "custom",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE,
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_bin= 31L,
  min_data_in_leaf= 20L,  #este ya es el valor default de LightGBM

  num_iterations= 9999L, # dejo libre la cantidad de arboles, zLightGBM se detiene solo
  num_leaves= 9999L, # dejo libre la cantidad de hojas, zLightGBM sabe cuando no hacer un split
  learning_rate= 1.0,  # se lo deja en 1.0 para que si el score esta por debajo de gradient_bound no se lo escale
    
  feature_fraction= 0.50, # un valor equilibrado, habra que probar alternativas ...
    
  canaritos= PARAM$qcanaritos, # fundamental en zLightGBM, aqui esta el control del overfitting
  gradient_bound= 0.1  # default de zLightGBM
)

Sys.time()

[1] "2025-11-07 20:02:49 UTC"

####  Entrenamiento del modelo

Aqui se ejecuta  *zLightGBM*
<br> no se utilizan hiperparámetros optimos para controlar el overfitting
<br> zLightGBM  sabe cuando no hacer un split
<br> si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina
<br>
<br> claramente el  min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [33]:
# entreno el modelo

modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)

Sys.time()

[1] "2025-11-07 20:10:32 UTC"

In [34]:
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
lgb.save(modelo_final, file="zmodelo.txt")

# grabo un dataset que tiene el detalle de los arboles de LightGBM
tb_arboles <- lgb.model.dt.tree(modelo_final)
fwrite(tb_arboles, file="tb_arboles.txt", sep="\t")

cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
cat("summary de las hojas de los arboles")
summary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])

Sys.time()

cantidad arbolitos= 256 
summary de las hojas de los arboles

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    3.0   311.8   498.5   502.8   685.2  1261.0 

[1] "2025-11-07 20:10:55 UTC"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [35]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$qcanaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

In [36]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "e007_prediccion.txt",
  sep= "\t"
)

### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [37]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./entrega", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)